In [1]:
from BDEx import get_geneset_data,ana, kg
import numpy as np
from scipy import stats 
import statsmodels.stats.multitest as multi
import pandas as pd

In [3]:
CCLE_expr = get_geneset_data.get_data("Depmap_21Q4","CCLE_expression")
CCLE_mutation = get_geneset_data.get_data("Depmap_21Q4","CCLE_mutation")
sample_info = get_geneset_data.get_data("Depmap_21Q4","sample_info")
CRISPR_gene_effect = get_geneset_data.get_data("Depmap_21Q4","CRISPR_gene_effect")
CRISPR_gene_dependency = get_geneset_data.get_data("Depmap_21Q4","CRISPR_gene_dependency")

In [ ]:
# Step 1: select the potential essential genes;
# Step 2: group the potential essential genes into common essential genes and context specific essential genes;
# Step 3: group the current drug target to the potential essential gene space;
# Step 4: comparison the essentiality between the mutated group and the WT group;
# Step 5: Find at least another evidence to support the relation

# Step 1: select the potential essential genes:

In [597]:
# Step 1: select the potential essential genes:

In [4]:
result = kg.select_all_potential_essential_genes(CRISPR_gene_dependency, 0.5)
gene_candidates = set(list(result.loc[result['probability'] >= 3/1054]['Gene'].values))

In [5]:
import pandas as pd
DDR_genes = pd.read_csv("/Users/guangrong/Documents/GitHub_project/fastqpi_BigGIM/KGs/cumulated_geneList/cumulated_geneset.csv",sep = ',')

In [15]:
genes = set(DDR_genes.loc[DDR_genes["gene_set"]=="DDR"]['Gene Symbol'])

In [18]:
x = result.loc[result['Gene'].isin(genes)]

In [19]:
DT = pd.read_csv("~/Documents/GitHub_project/fastqpi_BigGIM/KGs/Drug_Target/CTDMB_formated.csv")

In [22]:
merged_inner = pd.merge(left=x, right=DT, left_on='Gene', right_on='Target')


In [25]:
merged_inner.loc[:,['Gene','probability','Drug','Reference','Stage','Action']].to_csv("DDR_targets.csv")

# Step 2: group the potential essential genes into common essential genes and context specific essential genes;

In [ ]:
gene_candidates_common_q3 = set(result.loc[result['threshold_0.5']>0.75]['Gene'])
gene_candidates_common_q1 = set(result.loc[result['threshold_0.5']<0.25]['Gene']).intersection(gene_candidates)
gene_candidates_common_m = gene_candidates - (gene_candidates_common_q3) - (gene_candidates_common_q1)


In [75]:
print(len(gene_candidates_common_q1),len(gene_candidates_common_m),len(gene_candidates_common_q3))

4014 799 1359


# Step 3: group the current drug target to the potential essential gene space;

In [ ]:
## Develop the drug target KG and API

In [78]:
DT = pd.read_csv("~/Documents/GitHub_project/fastqpi_BigGIM/KGs/Drug_Target/CTDMB_formated.csv")

In [82]:
gene_candidates_common_q1_target = set(DT['Target']).intersection(gene_candidates_common_q1)
gene_candidates_common_q3_target = set(DT['Target']).intersection(gene_candidates_common_q3)
gene_candidates_common_m_target = set(DT['Target']).intersection(gene_candidates_common_m)

In [83]:
print(len(gene_candidates_common_q1_target),len(gene_candidates_common_m_target),len(gene_candidates_common_q3_target))

640 126 143


In [ ]:
# Plot 

# Step 4: comparison the essentiality between the mutated group and the WT group;


In [424]:
## Import data
#CCLE_expr = get_geneset_data.get_data("Depmap_21Q4","CCLE_expression")
CCLE_mutation = get_geneset_data.get_data("Depmap_21Q4","CCLE_mutation")
sample_info = get_geneset_data.get_data("Depmap_21Q4","sample_info")
CRISPR_gene_effect = get_geneset_data.get_data("Depmap_21Q4","CRISPR_gene_effect")
CRISPR_gene_dependency = get_geneset_data.get_data("Depmap_21Q4","CRISPR_gene_dependency")


https://ndownloader.figshare.com/files/31315930


/var/folders/w1/p1v0w3j165ggfdk4xpn0tzgh0000gp/T/ipykernel_92753/3293689844.py:3: DtypeWarning: Columns (3,19,22,27,28,29,30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  CCLE_mutation = get_geneset_data.get_data("Depmap_21Q4","CCLE_mutation")


https://ndownloader.figshare.com/files/31316011
https://ndownloader.figshare.com/files/31315996
https://ndownloader.figshare.com/files/31315990


In [454]:
CRISPR_gene_effect = get_geneset_data.get_data("Depmap_21Q4","CRISPR_gene_effect")

https://ndownloader.figshare.com/files/31315996


In [394]:
DDR_genes = pd.read_csv("/Users/guangrong/Documents/GitHub_project/fastqpi_BigGIM/KGs/cumulated_geneList/cumulated_geneset.csv",sep = ',')

In [520]:
Driver_genes = DDR_genes.loc[DDR_genes['gene_set'] == 'Driver gene']

In [589]:
#Analysis of mutation dependent essentiality
result = MDSLP("RHOBTB2", ['pancancer'],  sample_info,  CRISPR_gene_effect, CCLE_mutation, gene_candidates)

Number of samples with mutation: 28
Number of samples without mutation: 1015


In [ ]:
#result

# Step 5: Integrative analysis between mutation dependent essentiality and mutation dependent drug sensitivity

In [581]:
#Example run:
result_list = get_SL_with_multiple_evidence(gene, ['pancancer'], sample_info,  CRISPR_gene_effect, CCLE_mutation, gene_candidates)

In [584]:
all_result_list = []
all_mut_list = []
for gene in set(Driver_genes['Gene Symbol']):
    print(gene)
    result_list = get_SL_with_multiple_evidence(gene, ['pancancer'], sample_info,  CRISPR_gene_effect, CCLE_mutation, gene_candidates)
    mut_list = [gene]*len(result_list)
    all_result_list = all_result_list+result_list
    all_mut_list = all_mut_list + mut_list
result_df = pd.DataFrame({"Gene": all_mut_list, "SL": all_result_list})
result_df.to_csv("driver_high_conf_sl.csv")

SMARCA1
Number of samples with mutation: 39
Number of samples without mutation: 1007
PIK3R1
Number of samples with mutation: 62
Number of samples without mutation: 994
MET
Number of samples with mutation: 64
Number of samples without mutation: 984
ATR
Number of samples with mutation: 113
Number of samples without mutation: 948
FLNA
Number of samples with mutation: 124
Number of samples without mutation: 949
NUP93
Number of samples with mutation: 44
Number of samples without mutation: 1002
MAP3K1
Number of samples with mutation: 67
Number of samples without mutation: 980
PLXNB2
Number of samples with mutation: 113
Number of samples without mutation: 950
SF1
Number of samples with mutation: 43
Number of samples without mutation: 1003
AKT1
Number of samples with mutation: 25
Number of samples without mutation: 1020
CCND1
Number of samples with mutation: 11
Number of samples without mutation: 1032
MYCN
Number of samples with mutation: 23
Number of samples without mutation: 1019
STK11
Numbe

POLQ
Number of samples with mutation: 138
Number of samples without mutation: 926
PTPRD
Number of samples with mutation: 117
Number of samples without mutation: 941
ARID5B
Number of samples with mutation: 75
Number of samples without mutation: 979
KDM5C
Number of samples with mutation: 43
Number of samples without mutation: 1002
BCOR
Number of samples with mutation: 96
Number of samples without mutation: 963
KLF5
Number of samples with mutation: 26
Number of samples without mutation: 1019
ELF3
Number of samples with mutation: 35
Number of samples without mutation: 1011
ASXL1
Number of samples with mutation: 89
Number of samples without mutation: 963
ASXL2
Number of samples with mutation: 55
Number of samples without mutation: 993
FAT1
Number of samples with mutation: 252
Number of samples without mutation: 868
ALB
Number of samples with mutation: 24
Number of samples without mutation: 1020
LATS2
Number of samples with mutation: 61
Number of samples without mutation: 990
MAPK1
Number of

GPS2
Number of samples with mutation: 20
Number of samples without mutation: 1024
XPO1
Number of samples with mutation: 41
Number of samples without mutation: 1005
GNA13
Number of samples with mutation: 35
Number of samples without mutation: 1012
FOXQ1
Number of samples with mutation: 23
Number of samples without mutation: 1020
H3F3A
Number of samples with mutation: 8
Number of samples without mutation: 1034
IDH1
Number of samples with mutation: 24
Number of samples without mutation: 1018
GTF2I
Number of samples with mutation: 30
Number of samples without mutation: 1015
LATS1
Number of samples with mutation: 61
Number of samples without mutation: 986
HRAS
Number of samples with mutation: 21
Number of samples without mutation: 1021
RXRA
Number of samples with mutation: 19
Number of samples without mutation: 1023
MYH9
Number of samples with mutation: 120
Number of samples without mutation: 947
BAP1
Number of samples with mutation: 41
Number of samples without mutation: 1004
PPM1D
Number 